In [86]:
import torch
from torch import nn
import matplotlib.pyplot as plt

In [87]:
class Vocab:
    def __init__(self, tokens) -> None:
        if isinstance(tokens[0], list):
            tokens = [token for line in tokens for token in line]
        import collections
        counter = collections.Counter(tokens)
        tokens = [
            k for k, _ in sorted(
                counter.items(), key=lambda item: item[1], reverse=True)
        ]
        tokens.insert(0, '<unk>')
        self.tokens_indicates = {
            token: idx
            for idx, token in enumerate(tokens)
        }
        self.indicates_tokens = {
            v: k
            for k, v in self.tokens_indicates.items()
        }

    @property
    def unk(self):
        return 0

    def __len__(self):
        return len(self.tokens_indicates)

    def __getitem__(self, keys):
        if isinstance(keys, str):
            return self.tokens_indicates[keys]
        if isinstance(keys, list):
            return [self.__getitem__(key) for key in keys]
        if isinstance(keys, (torch.Tensor)):
            keys = keys.reshape(-1)
            return ''.join(self.indicates_tokens[int(keys[i])]
                           for i in range(keys.numel()))
        return self.indicates_tokens[keys]


def tokenize(lines: list, steps=32):
    import jieba
    import zhconv
    import re
    truncate = lambda sen, l: sen[:l] if len(sen) > l else sen + ['<pad>'] * (
        l - len(sen))
    en, zh = [], []
    for line in lines:
        sentence_en, sentence_zh, _ = line.split('\t')
        sentence_en = [
            i for i in re.sub('[^A-Za-z ]+', lambda m: f' {m.group()} ',
                              sentence_en).lower().split(' ')
            if i != '' and i != ' '
        ]
        sentence_zh = list(
            jieba.cut(zhconv.convert(sentence_zh, 'zh-cn'), cut_all=False))
        sentence_en = sentence_en + ['<eos>']
        sentence_zh = ['<bos>'] + sentence_zh + ['<eos>']
        sentence_en = truncate(sentence_en, steps)
        sentence_zh = truncate(sentence_zh, steps)
        en.append(sentence_en)
        zh.append(sentence_zh)
    return en, zh


def read_data():
    with open('../rnn/en_zh.trans.txt', 'r') as f:
        return f.readlines()


class _Dataset(torch.utils.data.Dataset):
    def __init__(self, data_raw) -> None:
        super().__init__()
        en, zh = data_raw
        self.vocab_en, self.vocab_zh = Vocab(en), Vocab(zh)
        self.corpus_en, self.corpus_zh = torch.tensor(
            self.vocab_en[en],
            dtype=torch.int64), torch.tensor(self.vocab_zh[zh],
                                               dtype=torch.int64)
        self.valid_len_en, self.valid_len_zh = (
            self.corpus_en != self.vocab_en['<pad>']).sum(
                dim=1), (self.corpus_zh != self.vocab_zh['<pad>']).sum(dim=1)
        self._len = len(self.corpus_en)
        del en, zh
        assert self._len == len(self.corpus_zh) == len(
            self.valid_len_en) == len(self.valid_len_zh)

    def __len__(self):
        return self._len

    def __getitem__(self, idx):
        return self.corpus_en[idx], self.valid_len_en[idx], self.corpus_zh[
            idx], self.valid_len_zh[idx]


In [88]:
class Encoder(nn.Module):
    def __init__(self,
                 vocab_size,
                 embed_size,
                 n_hiddens,
                 n_layers,
                 dropout=0.) -> None:
        super().__init__()
        # embedding layer similar as `one-hot` transformation.
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, n_hiddens, n_layers, dropout=dropout)

    def forward(self, x: torch.Tensor):
        # x.shape = (batch_size, n_steps)
        # embed.shape = (n_steps, batch_size, embed_size)
        embed = self.embedding(x.T.type(torch.int64))
        # output.shape = (n_steps, batch_size, n_hiddens)
        # state.shape = (n_layers, batch_size, n_hiddens)
        output, state = self.rnn(embed)
        return output, state


class Decoder(nn.Module):
    def __init__(self,
                 vocab_size,
                 embed_size,
                 n_hiddens,
                 n_layers,
                 dropout=0.) -> None:
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size + n_hiddens,
                          n_hiddens,
                          n_layers,
                          dropout=dropout)
        self.fc = nn.Linear(n_hiddens, vocab_size)

    def init_state(self, encode_outputs):
        encode_output, encode_state = encode_outputs
        return encode_state

    def forward(self, x, es: torch.Tensor):
        embed = self.embedding(x.T.type(torch.int64))
        # context using last layer of encode state
        # c.shape = (batch_size, n_hiddens)
        c = es[-1]
        c = c.repeat(embed.shape[0], 1, 1)
        embed_c = torch.cat((embed, c), -1)
        # decoder initial state as final `output_state` of encoder
        output, state = self.rnn(embed_c, es)
        output = self.fc(output)
        output = output.permute((1, 2, 0))
        # output.shape = (batch_size, n_steps, vocab_size)
        return output, state


class EncoderDecoder(nn.Module):
    def __init__(self,
                 vocab_size_source,
                 vocab_size_target,
                 embed_size,
                 n_hiddens,
                 n_layers,
                 dropout=0.) -> None:
        super().__init__()
        self.encoder = Encoder(vocab_size=vocab_size_source,
                               embed_size=embed_size,
                               n_hiddens=n_hiddens,
                               n_layers=n_layers,
                               dropout=dropout)
        self.decoder = Decoder(vocab_size=vocab_size_target,
                               embed_size=embed_size,
                               n_hiddens=n_hiddens,
                               n_layers=n_layers,
                               dropout=dropout)
    
    def forward(self, x, y):
        output = self.encoder(x)
        decode_state = self.decoder.init_state(output)
        output, _ = self.decoder(y, decode_state)
        return output

def loss_masking(loss, y, pad_indicate):
    mask = (y != pad_indicate).type(torch.float32)
    return (loss * mask).sum() / mask.sum()

def grad_clip(net: nn.Module, clip_val=1):
    params = [p for p in net.parameters() if p.requires_grad]
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > clip_val:
        for parm in params:
            parm.grad[:] *= clip_val / norm

In [89]:
# hyperparameter
EPOCHS = 50
LR = 1e-3
BATCH_SIZE = 128
STEPS = 19
EMBED_SIZE = 256
HIDDENS = 256
LAYERS = 3
DROPOUT = .3

In [94]:

dataset = _Dataset(tokenize(read_data(), steps=STEPS))
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=BATCH_SIZE,
                                         shuffle=True)
net = EncoderDecoder(vocab_size_source=len(dataset.vocab_en),
                     vocab_size_target=len(dataset.vocab_zh),
                     embed_size=EMBED_SIZE,
                     n_hiddens=HIDDENS,
                     n_layers=LAYERS,
                     dropout=DROPOUT)
loss_fn = nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.Adam(net.parameters(), lr=LR)
losses, loss = [], None
net.zero_grad()
for epoch in range(0, EPOCHS + 1):
    train_iter = iter(dataloader)
    for x, _, y, _ in train_iter:
        optimizer.zero_grad()
        # 1024 32 28
        output = net(x, y)
        loss = loss_fn(output, y)
        loss = loss_masking(loss, y, dataset.vocab_zh['<pad>'])
        with torch.no_grad():
            loss.backward()
            grad_clip(net)
            optimizer.step()
    print(f"\repoch: [{epoch}/{EPOCHS}", end='')
    losses.append(loss.detach().numpy())
    # print(f'loss: {loss: .6f}')
print()
plt.plot(losses)

epoch: [15/50

KeyboardInterrupt: 